https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html

Passpunkte

| 0  | 1  | 2  | 3  |
|----|----|----|----|
| 4  | 5  | 6  | 7  |
| 8  | 9 | 10 | 11 |
| 12 | 13 | 14 | 15 |

In [52]:
import time
import numpy as np
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix


In [53]:
passpunkte = [
    [0, 0, 0],
    [1, 0, 0],
    [2, 0, 0],
    [3, 0, 0],
    [0, 1, 0],
    [1, 1, 0],
    [2, 1, 0],
    [3, 1, 0],
    [0, 2, 0],
    [1, 2, 0],
    [2, 2, 0],
    [3, 2, 0],
    [0, 3, 0],
    [1, 3, 0],
    [2, 3, 0],
    [3, 3, 0]
]


In [54]:
# kamera, bildnr, passpunkt, bildkoordinatenx, y
messungen = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 2, 2, 0],
    [0, 0, 4, 0, 1],
    [0, 0, 5, 1, 1],
    [0, 0, 6, 2, 1],
    [0, 0, 8, 0, 2],
    [0, 0, 9, 1, 2],
    [0, 0, 10, 2, 2],
    [0, 1, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 1, 3, 2, 0],
    [0, 1, 5, 0, 1],
    [0, 1, 6, 1, 1],
    [0, 1, 7, 2, 1],
    [0, 1, 9, 0, 2],
    [0, 1, 10, 1, 2],
    [0, 1, 11, 2, 2],
    [0, 2, 4, 0, 0],
    [0, 2, 5, 1, 0],
    [0, 2, 6, 2, 0],
    [0, 2, 8, 0, 1],
    [0, 2, 9, 1, 1],
    [0, 2, 10, 2, 1],
    [0, 2, 12, 0, 2],
    [0, 2, 13, 1, 2],
    [0, 2, 14, 2, 2],
    [0, 3, 5, 0, 0],
    [0, 3, 6, 1, 0],
    [0, 3, 7, 2, 0],
    [0, 3, 9, 0, 1],
    [0, 3, 10, 1, 1],
    [0, 3, 11, 2, 1],
    [0, 3, 13, 0, 2],
    [0, 3, 14, 1, 2],
    [0, 3, 15, 2, 2],
    [0, 4, 0, 0, 0],
    [0, 4, 1, 1, 0],
    [0, 4, 2, 2, 0],
    [0, 4, 3, 3, 0],
    [0, 4, 4, 0, 1],
    [0, 4, 5, 1, 1],
    [0, 4, 6, 2, 1],
    [0, 4, 7, 3, 1],
    [0, 4, 8, 0, 2],
    [0, 4, 9, 1, 2],
    [0, 4, 10, 2, 2],
    [0, 4, 11, 3, 2],
    [0, 4, 12, 0, 3],
    [0, 4, 13, 1, 3],
    [0, 4, 14, 2, 3],
    [0, 4, 15, 3, 3]
])

In [55]:
# x,y,z, height, width
bild = [
    [1, 1, 1, 2, 2],
    [2, 1, 1, 2, 2],
    [1, 2, 1, 2, 2],
    [2, 2, 1, 2, 2],
    [1.5, 1.5, 1.5, 3, 3]
]


In [56]:
import time
import numpy as np
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix


In [57]:
camera_indices = messungen[:, 0]
image_indices = messungen[:, 1]
point_indices = messungen[:, 2]
points_2d = np.array([[i[3]/bild[i[1]][3]-0.5, i[4]/bild[i[1]][4]-0.5]
                     for i in messungen])
points_3d = np.array(passpunkte)
image_params = np.array([[0, 0, 0, i[0], i[1], i[2]] for i in bild])
camera_params = np.array([[0.5, 0, 0]])


In [58]:
n_cameras = camera_params.shape[0]
n_images = image_params.shape[0]
n_points = points_3d.shape[0]

n = n_cameras * 3 + 6 * n_images + 3 * n_points
m = 2 * points_2d.shape[0]

print("n_cameras: {}".format(n_cameras))
print("n_images: {}".format(n_images))
print("n_points: {}".format(n_points))
print("Total number of parameters: {}".format(n))
print("Total number of residuals: {}".format(m))

n_cameras: 1
n_images: 5
n_points: 16
Total number of parameters: 81
Total number of residuals: 104


In [59]:
def rotate(points, rot_vecs):
    """Rotate points by given rotation vectors.
    
    Rodrigues' rotation formula is used.
    """
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v


In [60]:
def project(points, camera_params, image_params):
    """Convert 3-D points to 2-D by projecting onto images."""
    points_proj = rotate(points, image_params[:, :3])
    points_proj += image_params[:, 3:6]
    points_proj = -points_proj[:, :2] / points_proj[:, 2, np.newaxis]
    f = camera_params[:, 0]
    k1 = camera_params[:, 1]
    k2 = camera_params[:, 2 ]
    n = np.sum(points_proj**2, axis=1)
    r = 1 + k1 * n + k2 * n**2
    points_proj *= (r * f)[:, np.newaxis]
    return points_proj


In [61]:
def fun(params, n_cameras, n_images, n_points, camera_indices, images_indices, point_indices, points_2d):
    """Compute residuals.

    `params` contains camera parameters and 3-D coordinates.
    """
    camera_params = params[:n_cameras * 3].reshape((n_cameras, 3))
    image_params = params[n_cameras * 3: n_cameras *
                          3+n_images*6].reshape((n_images, 6))
    points_3d = params[n_cameras*3+n_images*6:].reshape((n_points, 3))
    points_proj = project(
        points_3d[point_indices], camera_params[camera_indices], image_params[image_indices])
    return (points_proj - points_2d).ravel()


In [62]:
def bundle_adjustment_sparsity(n_cameras, n_images, n_points, camera_indices, image_indices, point_indices):
    m = image_indices.size * 2
    n = n_cameras * 3 + n_images * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)

    for s in range(3):
        A[2 * i, camera_indices * 3 + s] = 1
        A[2 * i + 1, camera_indices * 3 + s] = 1

    for s in range(6):
        A[2 * i, n_cameras * 3 + image_indices * 6 + s] = 1
        A[2 * i + 1, n_cameras * 3+ image_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, n_cameras * 3 + n_images * 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, n_cameras * 3 + n_images * 6 + point_indices * 3 + s] = 1

    return A


In [63]:
x0 = np.hstack((camera_params.ravel(), image_params.ravel(), points_3d.ravel()))
f0 = fun(x0, n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d)
x0.reshape(27,3)

array([[0.5, 0. , 0. ],
       [0. , 0. , 0. ],
       [1. , 1. , 1. ],
       [0. , 0. , 0. ],
       [2. , 1. , 1. ],
       [0. , 0. , 0. ],
       [1. , 2. , 1. ],
       [0. , 0. , 0. ],
       [2. , 2. , 1. ],
       [0. , 0. , 0. ],
       [1.5, 1.5, 1.5],
       [0. , 0. , 0. ],
       [1. , 0. , 0. ],
       [2. , 0. , 0. ],
       [3. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 1. , 0. ],
       [2. , 1. , 0. ],
       [3. , 1. , 0. ],
       [0. , 2. , 0. ],
       [1. , 2. , 0. ],
       [2. , 2. , 0. ],
       [3. , 2. , 0. ],
       [0. , 3. , 0. ],
       [1. , 3. , 0. ],
       [2. , 3. , 0. ],
       [3. , 3. , 0. ]])

In [64]:

A = bundle_adjustment_sparsity(
    n_cameras, n_images, n_points, camera_indices, image_indices,  point_indices)
t0 = time.time()
res = least_squares(fun, x0, jac_sparsity=A, verbose=2, x_scale='jac', ftol=1e-4, method='trf',
                    args=(n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d))
t1 = time.time()


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.3889e+02                                    1.91e+05    
       1              2         3.1888e+01      1.07e+02       5.43e+00       4.51e+03    
       2              3         4.2092e+00      2.77e+01       1.18e+01       7.27e-01    
       3              5         2.0186e+00      2.19e+00       2.42e+00       1.07e+01    
       4              6         3.3280e-01      1.69e+00       4.42e+00       6.33e+01    
       5              7         1.8579e-02      3.14e-01       1.43e+00       1.02e+01    
       6              8         4.2639e-03      1.43e-02       7.41e-01       2.27e+00    
       7              9         9.3914e-04      3.32e-03       4.13e-01       5.24e-01    
       8             10         2.0775e-04      7.31e-04       5.72e-01       1.20e-01    
       9             11         1.9487e-05      1.88e-04       3.37e-01       8.84e-02    

In [65]:
x0
np.array((res.x).round(3)).reshape(27,3)

array([[ 0.383,  0.   , -0.   ],
       [ 0.157, -0.157, -0.   ],
       [-1.07 , -1.07 ,  1.327],
       [ 0.157, -0.157, -0.   ],
       [ 0.87 , -1.07 ,  1.327],
       [ 0.157, -0.157, -0.   ],
       [-1.07 ,  0.87 ,  1.327],
       [ 0.157, -0.157, -0.   ],
       [ 0.87 ,  0.87 ,  1.327],
       [ 0.157, -0.157, -0.   ],
       [-0.1  , -0.1  ,  2.07 ],
       [ 1.711,  1.711, -1.279],
       [ 1.044,  2.984, -0.48 ],
       [-0.872,  3.008, -0.179],
       [-2.851,  3.112,  0.176],
       [ 2.984,  1.044, -0.48 ],
       [ 1.068,  1.068, -0.179],
       [-0.848,  1.092,  0.123],
       [-2.764,  1.116,  0.424],
       [ 3.008, -0.872, -0.179],
       [ 1.092, -0.848,  0.123],
       [-0.824, -0.824,  0.424],
       [-2.741, -0.801,  0.725],
       [ 3.112, -2.851,  0.176],
       [ 1.116, -2.764,  0.424],
       [-0.801, -2.741,  0.725],
       [-2.335, -2.335,  0.555]])

In [66]:
fun(res.x, n_cameras, n_images, n_points, camera_indices,
    image_indices, point_indices, points_2d)


array([ 5.67762781e-10,  5.67790037e-10,  1.83128024e-10, -1.01358577e-09,
       -3.71288555e-11,  1.95603200e-10, -1.01384179e-09,  1.83026959e-10,
       -6.40041765e-10, -6.39851367e-10,  8.43717873e-10, -6.66372290e-10,
        1.95582994e-10, -3.74675291e-11, -6.66122260e-10,  8.44139869e-10,
       -2.44964715e-10, -2.45235332e-10,  9.16050014e-11,  1.23919691e-09,
       -7.47209737e-11,  3.62766150e-10,  6.68836098e-10, -6.95292157e-10,
       -9.88164217e-10, -9.50261642e-10,  9.36880373e-10, -9.83814449e-10,
       -2.94949343e-10,  1.61316534e-10, -4.50736115e-11,  1.25253141e-11,
        9.31362201e-10,  9.94706539e-10, -1.20187732e-09, -1.65944425e-10,
        1.23939170e-09,  9.17919074e-11, -9.50212426e-10, -9.88485738e-10,
        1.26840760e-11, -4.51710891e-11,  3.62758934e-10, -7.43168294e-11,
       -9.83915285e-10,  9.36505444e-10,  9.94814231e-10,  9.31235097e-10,
       -6.95401181e-10,  6.68863187e-10,  1.61255701e-10, -2.94712976e-10,
       -1.66192948e-10, -

In [69]:
res.jac.toarray()

array([[-1.30628750e+00, -1.70638676e+00, -5.82351186e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.30628750e+00, -1.70638676e+00, -5.82351186e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.78435697e-10,  3.12487243e-10,  5.33224133e-10, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.30628750e+00,  9.47992660e-01,  1.79738027e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.30628750e+00,  1.70638677e+00,  5.82351189e+00, ...,
        -2.41607777e-01, -3.86943300e-02, -2.27069981e-01],
       [ 1.30628750e+00,  1.70638677e+00,  5.82351189e+00, ...,
        -3.86943504e-02, -2.41607698e-01, -2.27070067e-01]])